In [ ]:

!git clone https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
!git reset --hard
!git pull -f https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
import os
os.chdir('LMKBC-2023/')

In [ ]:
!pip install accelerate -U
#!pip install transformers

In [ ]:
!python src/run.py


In [ ]:
import torch

print(torch.cuda.is_available())

In [ ]:
dongyang-cs
